In [1]:
import requests
# re - модуль для работы с регулярными выражениями
import re
import time
import string
import random
import math
import pandas as pd
from urllib.parse import urlparse
from collections import Counter
from bs4 import BeautifulSoup
from pymystem3 import Mystem

# Задание №1

In [2]:
def crawl(urls):
    links_tree = set()
    for url in urls:
        if len(texts) >= 100:
            return
    
        if url in visited:
            continue
        visited.add(url)
            
        try:
            html = requests.get(url).content.decode()
        except:
            time.sleep(1)
            continue
            
        soup = BeautifulSoup(html)
        for script in soup(["script", "style"]):
            script.extract()
        text = soup.get_text(' ').strip().replace('\xa0', ' ').replace('\n', '').replace('\r', '').replace('\t', '')
        # re.sub() для замены всех ' +' на ' '
        text = re.sub(' +', ' ', text)
    
        if len(text.split(' ')) >= 1000:
            texts[url] = text
            
        parsed = urlparse(url)
        base = f"{parsed.scheme}://{parsed.netloc}"
        # re.findall() - найти все вхождения
        links = re.findall('''<a\s+(?:[^>]*?\s+)?href="([^"]*)"''', html) 
        
        for i, link in enumerate(links):    
            if not urlparse(link).netloc:    
                link_with_base = base + link    
                links[i] = link_with_base
                
        links_tree.update(set(filter(lambda x: 'mailto' not in x, links)))
    
    crawl(links_tree)

In [3]:
texts = {}
visited = set()
start = 'https://m.habr.com/ru/all/'

crawl([start])

In [4]:
index = {i:k for i, k in enumerate(texts)}

In [5]:
with open('index.txt', 'w') as f:
    for i in index:
        f.write(f"{i}: {index[i]}\n")

In [12]:
path = 'C:\\Users\\User\\Downloads\\InfoSearch\\InfoSearch_1\\index.txt'  # путь указывается индивидуально
open(path).read()

'0: https://m.habr.com/ru/post/543540/comments/\n1: https://m.habr.com/ru/company/itsoft/blog/543554/comments/\n2: https://m.habr.com/ru/post/543532/\n3: https://m.habr.com/ru/post/543550/\n4: https://m.habr.com/ru/post/543570/\n5: https://m.habr.com/ru/post/543546/comments/\n6: https://m.habr.com/ru/post/543534/\n7: https://m.habr.com/ru/post/543540/\n8: https://m.habr.com/ru/post/543586/\n9: https://m.habr.com/ru/post/543536/\n10: https://account.habr.com/info/confidential/\n11: https://m.habr.com/ru/company/vasexperts/blog/543300/\n12: https://m.habr.com/ru/company/itsoft/blog/543554/\n13: https://m.habr.com/ru/post/543550/comments/\n14: https://m.habr.com/ru/post/543542/comments/\n15: https://m.habr.com/ru/post/543552/\n16: https://m.habr.com/ru/post/543496/\n17: https://m.habr.com/ru/post/543570/comments/\n18: https://m.habr.com/ru/post/543580/\n19: https://account.habr.com/info/agreement\n20: https://m.habr.com/ru/company/audiomania/blog/541106/\n21: https://m.habr.com/ru/post/54

# Задание №2

In [13]:
#1. Из сохраненных документов выделить отдельные слова (токенизация)
#2. Лемматизировать токены
#Для оценки выполнения задания прислать:
#Txt файл по списком слов (Формат строк: <слово><\n>)
#Txt файл по списком лемматизированных токенов (Формат строки: <слово><пробел><токен1><пробел><токен2>.....<пробел><токенN><\n>)

In [14]:
# add: 1. записывать в Txt файл список слов для каждого сайта (Формат строк: <слово><\n>) 
# add: 2. записывать в Txt файл список лемматизированных токенов (Формат строки: <слово><пробел><токен1><пробел><токен2>.....<пробел><токенN><\n>)

In [15]:
# pymystem3.Mystem — морфологический анализатор русского языка с поддержкой снятия морфологической неоднозначности, 
# разработанный компанией «Яндекс». Программа работает на основе словаря и способна формировать морфологические гипотезы 
# о незнакомых словах
lemmatizer = Mystem()

In [17]:
texts_lemmatized = {}

for url in texts:
    text = texts[url]
    # maketrans(intab, outtab) возвращает таблицу перевода, которая сопоставляет каждый символ в строке intab с символом 
    # в той же позиции в строке outtab. Затем эта таблица передается функции translate().
    text = text.translate(str.maketrans('', '', string.punctuation)).replace('—', '')
    text = re.sub(' +', ' ', text)
    lemmas = lemmatizer.lemmatize(text)
    texts_lemmatized[url] = ''.join(lemmas).strip()
    # сохранить в файл тут

# Задание №3

In [18]:
docs = [index[k] for k in index]

In [19]:
# Создание инвертированного списка терминов (индекс)

reverse_index = {}

for url in texts_lemmatized:
    # пройдемся по списку лемматизированных токенов
    for word in texts_lemmatized[url].split(' '):
        # если слово не входит в reverse_index, то добавляем его туда
        if not word in reverse_index:
            reverse_index[word] = set([url])
        else:
            reverse_index[word].update([url])

In [20]:
# булев поиск по построенному индексу

logic = {
    '&': lambda x,y: x & y,
    '|': lambda x,y: x | y,
    '!': lambda x: set([doc for doc in docs if doc not in x])
}

def find(query):
    query = query.replace('(', ' ( ').replace(')', ' ) ').replace('!','! ').split()
    
    # слова, которые должны присутствовать на странице
    indexes_words = [i for i,x in enumerate(query) if not x in ['!', '&', '|']]
    for i in indexes_words:
        query[i] = reverse_index[query[i]]
    
    # слова, которые не должны присутствовать на странице
    indexes_not = [i for i,x in enumerate(query) if x == '!']
    for i in indexes_not:
        x = query[i+1]
        query[i+1] = logic['!'](x)
        query.pop(i)
    
    # фразы, которые должны присутствовать на странице
    indexes_and = [i for i,x in enumerate(query) if x == '&']
    for i in indexes_and:
        x = query[i-1]
        y = query[i+1]
        query[i-1] = logic['&'](x,y)
        # list.pop([i]) - Возвращает элемент [на указанной позиции], удаляя его из списка.
        query.pop(i)
        query.pop(i)
    
    # хотя бы одно из указанных тут слов должно присутствовать на странице
    indexes_or = [i for i,x in enumerate(query) if x == '|']
    for i in indexes_or:
        x = query[i-1]
        y = query[i+1]
        query[i-1] = logic['|'](x,y)
        query.pop(i)
        query.pop(i)
    
    return query[0]

In [32]:
# random.choice() - возвращает случайный элемент из указанной последовательности
x1 = random.choice(list(reverse_index.keys()))
x2 = random.choice(list(reverse_index.keys()))
x3 = random.choice(list(reverse_index.keys()))

In [33]:
query = f'{x1} | {x2} & !{x3}'
print(query)

ГОСТ121005 | she & !числовой


In [34]:
find(query)

{'https://en.wikipedia.org/wiki/Assembly_language#Assembler',
 'https://files.stroyinf.ru/Data1/10/10678/index.htm'}

# Задание №4

In [35]:
#Задание:
#Подсчитать tf каждого термина
#Подсчитать idf
#Подсчитать tf-idf
#Для оценки выполнения задания прислать:
#ссылку на рабочую версию кода в репозитории
#Txt файл по списком терминов (Формат строк: <термин><пробел><idf><пробел><tf-idf><\n>)

In [ ]:
# add: Txt файл по списком терминов (Формат строк: <термин><пробел><idf><пробел><tf-idf><\n>)

In [15]:
tfs = {}

for word in reverse_index:
    for doc in docs:
        tokens = texts[doc].split()
        tf = tokens.count(word) / len(tokens)
        if not word in tfs:
            tfs[word] = {}
        tfs[word][doc] = tf

In [16]:
pd.DataFrame(tfs).T

,https://m.habr.com/ru/post/494504/,https://m.habr.com/ru/company/wd/blog/493378/,https://m.habr.com/ru/post/494460/,https://m.habr.com/ru/post/494474/,https://www.youtube.com/channel/UCd_sTwKqVrweTt4oAKY5y4w,https://habr.com/ru/all/?mobile=no,https://m.habr.com/ru/company/otus/blog/494480/,https://m.habr.com/ru/company/wd/blog/493378/comments/,https://m.habr.com/ru/company/pt/blog/494472/,https://twitter.com/habr_com,...,http://admixer.by/,https://habr.com/ru/hub/soft/,https://netacad.com,https://habr.com/ru/post/494492/#comments,https://habr.com/ru/hub/infosecurity/,https://habr.com/ru/company/wd/blog/493378/?mobile=no,https://github.com/reduxjs/redux,https://habr.com/ru/search/?q=%5B%D0%BE%D0%B1%D1%83%D1%87%D0%B5%D0%BD%D0%B8%D0%B5%5D&target_type=posts,https://habr.com/ru/post/494474/,https://habr.com/ru/post/494478/#habracut
имитированый,0.000000,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,...,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0,0.0,0.0,0.000000
Cisco,0.009498,0.0,0.0,0.0,0.0,0.000649,0.0,0.0,0.0,0.0,...,0.0,0.0,0.011459,0.0,0.000997,0.0,0.0,0.0,0.0,0.000000
идентичный,0.000000,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,...,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0,0.0,0.0,0.000000
натуральный,0.000000,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,...,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0,0.0,0.0,0.000000
хабр,0.000000,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,...,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0,0.0,0.0,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
QGroundControl,0.000000,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,...,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0,0.0,0.0,0.000505
PixHawk,0.000000,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,...,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0,0.0,0.0,0.000000
VOLO,0.000000,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,...,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0,0.0,0.0,0.000505
Wunder,0.000000,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,...,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0,0.0,0.0,0.000505


In [17]:
idfs = {}

for word in reverse_index:
    idfs[word] = math.log(len(docs) / len(reverse_index[word]))

In [18]:
pd.DataFrame(idfs,index=['']).T

,
имитированый,3.218876
Cisco,2.120264
идентичный,1.897120
натуральный,2.659260
хабр,0.235722
...,...
QGroundControl,4.605170
PixHawk,4.605170
VOLO,4.605170
Wunder,4.605170


In [19]:
tfidfs = {}

for word in tfs:
    for doc in tfs[word]:
        tfidf = tfs[word][doc] * idfs[word]
        if not word in tfidfs:
            tfidfs[word] = {}
        tfidfs[word][doc] = tfidf

In [20]:
pd.DataFrame(tfidfs).T

,https://m.habr.com/ru/post/494504/,https://m.habr.com/ru/company/wd/blog/493378/,https://m.habr.com/ru/post/494460/,https://m.habr.com/ru/post/494474/,https://www.youtube.com/channel/UCd_sTwKqVrweTt4oAKY5y4w,https://habr.com/ru/all/?mobile=no,https://m.habr.com/ru/company/otus/blog/494480/,https://m.habr.com/ru/company/wd/blog/493378/comments/,https://m.habr.com/ru/company/pt/blog/494472/,https://twitter.com/habr_com,...,http://admixer.by/,https://habr.com/ru/hub/soft/,https://netacad.com,https://habr.com/ru/post/494492/#comments,https://habr.com/ru/hub/infosecurity/,https://habr.com/ru/company/wd/blog/493378/?mobile=no,https://github.com/reduxjs/redux,https://habr.com/ru/search/?q=%5B%D0%BE%D0%B1%D1%83%D1%87%D0%B5%D0%BD%D0%B8%D0%B5%5D&target_type=posts,https://habr.com/ru/post/494474/,https://habr.com/ru/post/494478/#habracut
имитированый,0.000000,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,...,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0,0.0,0.0,0.000000
Cisco,0.020139,0.0,0.0,0.0,0.0,0.001377,0.0,0.0,0.0,0.0,...,0.0,0.0,0.024296,0.0,0.002113,0.0,0.0,0.0,0.0,0.000000
идентичный,0.000000,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,...,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0,0.0,0.0,0.000000
натуральный,0.000000,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,...,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0,0.0,0.0,0.000000
хабр,0.000000,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,...,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0,0.0,0.0,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
QGroundControl,0.000000,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,...,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0,0.0,0.0,0.002323
PixHawk,0.000000,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,...,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0,0.0,0.0,0.000000
VOLO,0.000000,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,...,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0,0.0,0.0,0.002323
Wunder,0.000000,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,...,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0,0.0,0.0,0.002323


# Задание №5

In [ ]:
#Задание:
#1. Разработать поисковую систему на основе векторного поиска по построенному индексу
#Для оценки выполнения задания прислать: ссылку на рабочую версию кода в репозитории сообщить, 
#что будете проводить демонстрацию лично или прислать ссылку на сайт, видео

In [21]:
docs_length = {}
for row in pd.DataFrame(tfidfs).iterrows():
    sum_len = 0
    for word in row[1]:
        sum_len += word**2
    docs_length[row[0]] = math.sqrt(sum_len)

In [22]:
query = "правовая защита контента"
query = ''.join(lemmatizer.lemmatize(query)).strip()

tfs_query = {}
for word in query.split():
    for doc in docs:
        tokens = texts[doc].split()
        tf = tokens.count(word) / len(tokens)
        if not word in tfs_query:
            tfs_query[word] = {}
        tfs_query[word][doc] = tf

In [23]:
idfs_query = {}

for word in query.split():
    idfs_query[word] = math.log(len(docs) / len(reverse_index[word]))

In [24]:
tfidfs_query = {}

for word in tfs_query:
    for doc in tfs_query[word]:
        tfidf = tfs_query[word][doc] * idfs_query[word]
        if not word in tfidfs_query:
            tfidfs_query[word] = {}
        tfidfs_query[word][doc] = tfidf

In [25]:
tfidfs_query_flat = pd.DataFrame(tfidfs_query).T.sum().to_dict()

In [26]:
sum_len = 0
for doc in tfidfs_query_flat:
    sum_len += tfidfs_query_flat[doc]**2
query_length = math.sqrt(sum_len)

In [27]:
cos_sim = {}
for doc in docs:
    for word in query.split(' '):
        cs = ((pd.Series(tfidfs_query_flat) * tfs[word][doc]).sum()) / (docs_length[doc] * query_length)
    cos_sim[doc] = cs

In [28]:
Counter(cos_sim).most_common(5)

[('https://m.habr.com/ru/post/494488/', 0.3550683322376472),
 ('https://tmtm.ru/megapost', 0.036782017144031266),
 ('https://m.habr.com/ru/company/wd/blog/493378/comments/',
  0.016721698695944514),
 ('https://habr.com/ru/hub/gadgets/', 0.01652857595533733),
 ('https://m.habr.com/ru/company/pt/blog/494486/', 0.012840283695295754)]